# Federated 2d image classification with MONAI

## Introduction

This tutorial shows how to deploy in Fed-BioMed the 2d image classification example provided in the project MONAI (https://monai.io/):

https://github.com/Project-MONAI/tutorials/blob/master/2d_classification/mednist_tutorial.ipynb

Being MONAI based on PyTorch, the deployment within Fed-BioMed follows seamlessy the same general structure of general PyTorch models. 

Following the MONAI example, this tutorial is based on the MedNIST dataset:

https://github.com/Project-MONAI/MONAI/blob/master/examples/notebooks/mednist_tutorial.ipynb.

## Creating MedNIST nodes

MedNIST provides an artificial 2d classification dataset created by gathering different medical imaging datasets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license.

To proceed with the tutorial, we created an iid partitioning of the MedNIST dataset between 3 clients. Each client has 3000 image samples for each class. The training partitions are availables at the following link:

https://drive.google.com/file/d/1vLIcBdtdAhh6K-vrgCFy_0Y55dxOWZwf/view

The dataset owned by each client has structure:


└── client_*/

    ├── AbdomenCT/
    
    └── BreastMRI/
    
    └── CXR/
    
    └── ChestCT/
    
    └── Hand/
    
    └── HeadCT/   

To create the federated dataset, we follow the standard procedure for node creation/population of Fed-BioMed. 
After activating the fedbiomed network with the commands

`source ./scripts/fedbiomed_environment network`

and 

`./scripts/fedbiomed_run network`

we create a first node by using the commands

`source ./scripts/fedbiomed_environment node`

`./scripts/fedbiomed_run node start`

We then poulate the node with the data of first client:

`./scripts/fedbiomed_run node add`

We select option 3 (images) to add MedNIST partition of client 1, by just picking the folder of client 1. 
Assign tag `mednist` to the data when asked.

We can further check that the data has been added by executing `./scripts/fedbiomed_run node list`

Following the same procedure, we create the other two nodes with the datasets of client 2 and client 3 respectively.


## Running Fed-BioMed Researcher

We are now ready to start the reseracher enviroment with the command `source ./scripts/fedbiomed_environment researcher`, and open the Jupyter notebook. 

In [1]:
%load_ext autoreload
%autoreload 2

We can first quesry the network for the mednist dataset. In this case, the nodes are sharing the respective partitions unsing the same tag `mednist`:

In [1]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)


2022-01-06 19:29:58,786 fedbiomed INFO - Component environment:
2022-01-06 19:29:58,787 fedbiomed INFO - - type = ComponentType.RESEARCHER
2022-01-06 19:29:59,143 fedbiomed INFO - Messaging researcher_ccf4bfd5-a09e-4e86-b698-d656c390b98f successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7fbd8854dd90>
2022-01-06 19:29:59,170 fedbiomed INFO - Listing available datasets in all nodes... 
2022-01-06 19:29:59,172 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / DEBUG - Message received: {'researcher_id': 'researcher_ccf4bfd5-a09e-4e86-b698-d656c390b98f', 'command': 'list'}
2022-01-06 19:29:59,183 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / DEBUG - Message received: {'researcher_id': 'researcher_ccf4bfd5-a09e-4e86-b698-d656c390b98f', 'command': 'list'}
2022-01-06 19:29:59,185 fedbiomed INFO - log from: node_0341dbee-4250-4f28-8f05-fc0a98bde6b7 / DEBUG - Message received: {'researcher_id': '

{'node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a': [{'name': 'mednist',
   'data_type': 'images',
   'tags': ['mednist'],
   'description': '',
   'shape': [18000, 3, 64, 64]}],
 'node_30bcc00b-f402-4658-982c-6642390eb80a': [{'name': 'mednist',
   'data_type': 'images',
   'tags': ['mednist'],
   'description': '',
   'shape': [16954, 3, 64, 64]}],
 'node_0341dbee-4250-4f28-8f05-fc0a98bde6b7': [{'name': 'mednist',
   'data_type': 'images',
   'tags': ['mednist'],
   'description': '',
   'shape': [18000, 3, 64, 64]}]}

## Create an experiment to train a model on the data found

The code for network and data loader of the MONAI tutorial can now be deployed in Fed-BioMed.
We first import the necessary modules from `fedbiomed` and `monai` libraries:

In [2]:
from fedbiomed.researcher.environ import environ
import tempfile
import os

tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+os.sep)
model_file = os.path.join(tmp_dir_model.name, 'class_export_mednist.py')

In [3]:
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import decollate_batch
from monai.metrics import ROCAUCMetric
from monai.networks.nets import DenseNet121
from monai.transforms import (
    Activations,
    AddChannel,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    EnsureType,
)
from monai.utils import set_determinism

We can now define the training plan. Note that we can simply use the standard `TorchTrainingPlan` natively provided in Fed-BioMed. We reuse the `MedNISTDataset` data loader defined in the original MONAI tutorial, which is returned by the method `training_data`, which also implements the data parsing from the nodes `dataset_path`. Following the MONAI tutorial, the model is the `DenseNet121`.

In [4]:
%%writefile "$model_file"

import os
import numpy as np
import torch
import torch.nn as nn
from fedbiomed.common.torchnn import TorchTrainingPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import decollate_batch
from monai.metrics import ROCAUCMetric
from monai.networks.nets import DenseNet121
from monai.transforms import (
    Activations,
    AddChannel,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    EnsureType,
)
from monai.utils import set_determinism


# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, kwargs):
        super(MyTrainingPlan, self).__init__()
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["import numpy as np",
                "import os",
                "from torch.utils.data import DataLoader",
                "from monai.apps import download_and_extract",
                "from monai.config import print_config",
                "from monai.data import decollate_batch",
                "from monai.metrics import ROCAUCMetric",
                "from monai.networks.nets import DenseNet121",
                "from monai.transforms import ( Activations, AddChannel, AsDiscrete, Compose, LoadImage, RandFlip, RandRotate, RandZoom, ScaleIntensity, EnsureType, )",
                "from monai.utils import set_determinism",]
        self.add_dependency(deps)
         
        self.num_class =  kwargs['num_class']  
        self.model = DenseNet121(spatial_dims=2, in_channels=1,
                    out_channels = self.num_class)
        
        self.loss_function = torch.nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    class MedNISTDataset(torch.utils.data.Dataset):
            def __init__(self, image_files, labels, transforms):
                self.image_files = image_files
                self.labels = labels
                self.transforms = transforms

            def __len__(self):
                return len(self.image_files)

            def __getitem__(self, index):
                return self.transforms(self.image_files[index]), self.labels[index]
    
    def parse_data(self, path):
        print(self.dataset_path)
        class_names = sorted(x for x in os.listdir(path)
                     if os.path.isdir(os.path.join(path, x)))
        num_class = len(class_names)
        image_files = [
                        [
                            os.path.join(path, class_names[i], x)
                            for x in os.listdir(os.path.join(path, class_names[i]))
                        ]
                        for i in range(num_class)
                      ]
        
        return image_files, num_class
    
    def training_data(self, batch_size = 48):
        self.image_files, num_class = self.parse_data(self.dataset_path)
        
        if self.num_class!=num_class:
                raise Exception('number of available classes does not match declared classes')
        
        num_each = [len(self.image_files[i]) for i in range(self.num_class)]
        image_files_list = []
        image_class = []
        
        for i in range(self.num_class):
            image_files_list.extend(self.image_files[i])
            image_class.extend([i] * num_each[i])
        num_total = len(image_class)
        
        
        length = len(image_files_list)
        indices = np.arange(length)
        np.random.shuffle(indices)

        val_split = int(1. * length) 
        train_indices = indices[:val_split]

        train_x = [image_files_list[i] for i in train_indices]
        train_y = [image_class[i] for i in train_indices]


        train_transforms = Compose(
            [
                LoadImage(image_only=True),
                AddChannel(),
                ScaleIntensity(),
                RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
                RandFlip(spatial_axis=0, prob=0.5),
                RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
                EnsureType(),
            ]
        )

        val_transforms = Compose(
            [LoadImage(image_only=True), AddChannel(), ScaleIntensity(), EnsureType()])

        y_pred_trans = Compose([EnsureType(), Activations(softmax=True)])
        y_trans = Compose([EnsureType(), AsDiscrete(to_onehot=num_class)])

        print(
            f"Training count: {len(train_x)}")
        
        
        train_ds = self.MedNISTDataset(train_x, train_y, train_transforms)
        train_loader = torch.utils.data.DataLoader(
            train_ds, batch_size, shuffle=True)
        
        return train_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = self.loss_function(output, target)
        return loss


Writing /home/ybouilla/fedbiomed/var/tmp/tmpo8drwl14/class_export_mednist.py


We now set the model and training parameters. Note that we use only 1 epoch for this experiment, and perform the training on ~26% of the locally available training data.

In [5]:
model_args = {'num_class':6,}

training_args = {
    'batch_size': 20, 
    'lr': 1e-5, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum':250 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

The experiment can be now defined, by providing the `mednist` tag, and running the local training on nodes with model defined in `model_path`, standard `aggregator` (FedAvg) and `client_selection_strategy` (all nodes used). Federated learning is going to be perfomed through 3 optimization rounds.

In [6]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['mednist']
rounds = 3

exp = Experiment(tags=tags,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None
                )

2022-01-06 19:30:34,669 fedbiomed INFO - Searching dataset with data tags: ['mednist'] for all nodes
2022-01-06 19:30:34,671 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / DEBUG - Message received: {'researcher_id': 'researcher_ccf4bfd5-a09e-4e86-b698-d656c390b98f', 'tags': ['mednist'], 'command': 'search'}
2022-01-06 19:30:34,671 fedbiomed INFO - log from: node_0341dbee-4250-4f28-8f05-fc0a98bde6b7 / DEBUG - Message received: {'researcher_id': 'researcher_ccf4bfd5-a09e-4e86-b698-d656c390b98f', 'tags': ['mednist'], 'command': 'search'}
2022-01-06 19:30:34,672 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / DEBUG - Message received: {'researcher_id': 'researcher_ccf4bfd5-a09e-4e86-b698-d656c390b98f', 'tags': ['mednist'], 'command': 'search'}
2022-01-06 19:30:44,682 fedbiomed INFO - Node selected for training -> node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a
2022-01-06 19:30:44,685 fedbiomed INFO - Node selected for training -> node_0341dbee-4250-

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the clients

In [7]:
exp.run()

2022-01-06 19:37:42,385 fedbiomed INFO - Sampled nodes in round 0 ['node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a', 'node_0341dbee-4250-4f28-8f05-fc0a98bde6b7', 'node_30bcc00b-f402-4658-982c-6642390eb80a']
2022-01-06 19:37:42,390 fedbiomed INFO - Send message to node node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a - {'researcher_id': 'researcher_ccf4bfd5-a09e-4e86-b698-d656c390b98f', 'job_id': '2f5ac007-c792-442b-a7a7-4cc24ea9a186', 'training_args': {'batch_size': 20, 'lr': 1e-05, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'num_class': 6}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/06/my_model_3741fb7c-a0e8-48fe-82de-48ef82904df1.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/06/aggregated_params_init_365d8c09-1c12-485c-b7eb-4367e7305781.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a': ['dataset_9088096e-ff15-4339-ada6-65974ba24a1c']}}
2022-01-06 19:37:42,393 fedbiomed

2022-01-06 19:37:43,276 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / INFO - {'monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x7f7a5e7baf40>, 'batch_size': 20, 'lr': 1e-05, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 250}
2022-01-06 19:37:43,278 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / DEBUG - Dataset_path/home/ybouilla/fedbiomed/notebooks/data/mednist/client_2
2022-01-06 19:37:46,824 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / INFO - Train Epoch: 1 [0/16954 (0%)]	Loss: 1.769837
2022-01-06 19:37:46,913 fedbiomed INFO - log from: node_0341dbee-4250-4f28-8f05-fc0a98bde6b7 / INFO - Train Epoch: 1 [0/18000 (0%)]	Loss: 1.816945
2022-01-06 19:37:47,121 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / INFO - Train Epoch: 1 [0/18000 (0%)]	Loss: 1.787506
2022-01-06 19:38:36,367 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / INFO - Train Epoch: 1

2022-01-06 19:51:31,564 fedbiomed INFO - log from: node_0341dbee-4250-4f28-8f05-fc0a98bde6b7 / INFO - Train Epoch: 1 [3600/18000 (20%)]	Loss: 0.782915
2022-01-06 19:51:44,564 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / INFO - Train Epoch: 1 [3400/18000 (19%)]	Loss: 0.612654
2022-01-06 19:52:05,176 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / INFO - Train Epoch: 1 [3400/16954 (20%)]	Loss: 0.538178
2022-01-06 19:52:20,741 fedbiomed INFO - log from: node_0341dbee-4250-4f28-8f05-fc0a98bde6b7 / INFO - Train Epoch: 1 [3800/18000 (21%)]	Loss: 0.741194
2022-01-06 19:52:32,183 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / INFO - Train Epoch: 1 [3600/18000 (20%)]	Loss: 0.536325
2022-01-06 19:52:59,882 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / INFO - Train Epoch: 1 [3600/16954 (21%)]	Loss: 0.621068
2022-01-06 19:53:17,083 fedbiomed INFO - log from: node_0341dbee-4250-4f28-8f05-fc0a98bde6b7 /

2022-01-06 19:57:39,827 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / DEBUG - Message received: {'researcher_id': 'researcher_ccf4bfd5-a09e-4e86-b698-d656c390b98f', 'job_id': '2f5ac007-c792-442b-a7a7-4cc24ea9a186', 'training_args': {'batch_size': 20, 'lr': 1e-05, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'num_class': 6}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/06/my_model_3741fb7c-a0e8-48fe-82de-48ef82904df1.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/06/aggregated_params_bc671c9c-54b8-4727-96f5-dbb2aeaeeb5b.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a': ['dataset_9088096e-ff15-4339-ada6-65974ba24a1c']}}
2022-01-06 19:57:39,829 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_ccf4bfd5-a09e-4e86-b698-d656c390b98f', 'job_id': '2f5ac007-c792-442b-a7a7

2022-01-06 20:00:27,547 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / INFO - Train Epoch: 1 [1000/18000 (6%)]	Loss: 0.360890
2022-01-06 20:00:38,634 fedbiomed INFO - log from: node_0341dbee-4250-4f28-8f05-fc0a98bde6b7 / INFO - Train Epoch: 1 [1000/18000 (6%)]	Loss: 0.573974
2022-01-06 20:00:48,731 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / INFO - Train Epoch: 1 [1000/16954 (6%)]	Loss: 0.335039
2022-01-06 20:01:03,748 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / INFO - Train Epoch: 1 [1200/18000 (7%)]	Loss: 0.443353
2022-01-06 20:01:18,684 fedbiomed INFO - log from: node_0341dbee-4250-4f28-8f05-fc0a98bde6b7 / INFO - Train Epoch: 1 [1200/18000 (7%)]	Loss: 0.253346
2022-01-06 20:01:26,381 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / INFO - Train Epoch: 1 [1200/16954 (7%)]	Loss: 0.267557
2022-01-06 20:01:44,236 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / INFO 

2022-01-06 20:11:45,342 fedbiomed INFO - log from: node_0341dbee-4250-4f28-8f05-fc0a98bde6b7 / INFO - Train Epoch: 1 [4600/18000 (26%)]	Loss: 0.206550
2022-01-06 20:11:52,177 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / INFO - Train Epoch: 1 [4800/18000 (27%)]	Loss: 0.218345
2022-01-06 20:12:08,303 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / INFO - Train Epoch: 1 [4600/16954 (27%)]	Loss: 0.325356
2022-01-06 20:12:19,334 fedbiomed INFO - log from: node_0341dbee-4250-4f28-8f05-fc0a98bde6b7 / INFO - Train Epoch: 1 [4800/18000 (27%)]	Loss: 0.215443
2022-01-06 20:12:27,861 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / DEBUG - Reached 250 batches for this epoch, ignore remaining data
2022-01-06 20:12:28,790 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / INFO - results uploaded successfully 
2022-01-06 20:12:32,788 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / INFO - 

2022-01-06 20:12:46,782 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / DEBUG - Message received: {'researcher_id': 'researcher_ccf4bfd5-a09e-4e86-b698-d656c390b98f', 'job_id': '2f5ac007-c792-442b-a7a7-4cc24ea9a186', 'training_args': {'batch_size': 20, 'lr': 1e-05, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'num_class': 6}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/06/my_model_3741fb7c-a0e8-48fe-82de-48ef82904df1.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/06/aggregated_params_6d142ddc-0a8f-4ca3-9c94-c790879157f4.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_30bcc00b-f402-4658-982c-6642390eb80a': ['dataset_1457517a-ff70-4d89-95d6-e5e16208e917']}}
2022-01-06 20:12:46,820 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_ccf4bfd5-a09e-4e86-b698-d656c390b98f', 'job_id': '2f5ac007-c792-442b-a7a7

2022-01-06 20:19:56,105 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / INFO - Train Epoch: 1 [2400/18000 (13%)]	Loss: 0.079549
2022-01-06 20:20:05,931 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / INFO - Train Epoch: 1 [2400/16954 (14%)]	Loss: 0.190527
2022-01-06 20:20:17,857 fedbiomed INFO - log from: node_0341dbee-4250-4f28-8f05-fc0a98bde6b7 / INFO - Train Epoch: 1 [2400/18000 (13%)]	Loss: 0.137602
2022-01-06 20:20:36,363 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a / INFO - Train Epoch: 1 [2600/18000 (14%)]	Loss: 0.097215
2022-01-06 20:20:43,046 fedbiomed INFO - log from: node_30bcc00b-f402-4658-982c-6642390eb80a / INFO - Train Epoch: 1 [2600/16954 (15%)]	Loss: 0.134369
2022-01-06 20:20:51,721 fedbiomed INFO - log from: node_0341dbee-4250-4f28-8f05-fc0a98bde6b7 / INFO - Train Epoch: 1 [2600/18000 (14%)]	Loss: 0.195798
2022-01-06 20:21:16,440 fedbiomed INFO - log from: node_b4eb76b3-54fe-42e4-8f01-501a4f1a179a /

## Testing


Once the federated model is obtained, it is possible to test it locally on an independent testing partition.
The test dataset is available at this link:

https://drive.google.com/file/d/1YbwA0WitMoucoIa_Qao7IC1haPfDp-XD/

In [8]:
!pip install gdown

In [9]:
import os
import shutil
import tempfile
import PIL
import torch
import numpy as np
from sklearn.metrics import classification_report

from monai.config import print_config
from monai.data import decollate_batch
from monai.metrics import ROCAUCMetric
from monai.networks.nets import DenseNet121
import zipfile
from monai.transforms import (
    Activations,
    AddChannel,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    EnsureType,
)
from monai.utils import set_determinism

print_config()

MONAI version: 0.8.0
Numpy version: 1.21.2
Pytorch version: 1.8.1+cu102
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 714d00dffe6653e21260160666c4c201ab66511b

Optional dependencies:
Pytorch Ignite version: 0.4.6
Nibabel version: 3.2.1
scikit-image version: 0.19.0
Pillow version: 8.4.0
Tensorboard version: 2.7.0
gdown version: 4.2.0
TorchVision version: 0.9.1+cu102
tqdm version: 4.62.3
lmdb version: 1.2.1
psutil version: 5.8.0
pandas version: 1.3.5
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



Download the testing dataset on the local temporary folder.

In [10]:
import gdown
import zipfile

resource = "https://drive.google.com/uc?id=1YbwA0WitMoucoIa_Qao7IC1haPfDp-XD"
base_dir = tmp_dir_model.name 
test_file = os.path.join(base_dir, "MedNIST_testing.zip")

gdown.download(resource, test_file, quiet=False)

zf = zipfile.ZipFile(test_file)

for file in zf.infolist():
    zf.extract(file, base_dir)
    
data_dir = os.path.join(base_dir, "MedNIST_testing")

Downloading...
From: https://drive.google.com/uc?id=1YbwA0WitMoucoIa_Qao7IC1haPfDp-XD
To: /home/ybouilla/fedbiomed/var/tmp/tmpo8drwl14/MedNIST_testing.zip
100%|██████████████████████████████████████| 9.50M/9.50M [00:07<00:00, 1.31MB/s]


Parse the data and create the testing data loader:

In [11]:
class_names = sorted(x for x in os.listdir(data_dir)
                     if os.path.isdir(os.path.join(data_dir, x)))
num_class = len(class_names)
image_files = [
    [
        os.path.join(data_dir, class_names[i], x)
        for x in os.listdir(os.path.join(data_dir, class_names[i]))
    ]
    for i in range(num_class)
]

num_each = [len(image_files[i]) for i in range(num_class)]
image_files_list = []

image_class = []
for i in range(num_class):
    image_files_list.extend(image_files[i])
    image_class.extend([i] * num_each[i])
num_total = len(image_class)
image_width, image_height = PIL.Image.open(image_files_list[0]).size

print(f"Total image count: {num_total}")
print(f"Image dimensions: {image_width} x {image_height}")
print(f"Label names: {class_names}")
print(f"Label counts: {num_each}")

Total image count: 6000
Image dimensions: 64 x 64
Label names: ['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'Hand', 'HeadCT']
Label counts: [1000, 1000, 1000, 1000, 1000, 1000]


In [12]:
length = len(image_files_list)
indices = np.arange(length)
np.random.shuffle(indices)


test_split = int(0.1 * length)
test_indices = indices[:test_split]

test_x = [image_files_list[i] for i in test_indices]
test_y = [image_class[i] for i in test_indices]

val_transforms = Compose(
    [LoadImage(image_only=True), AddChannel(), ScaleIntensity(), EnsureType()])

y_pred_trans = Compose([EnsureType(), Activations(softmax=True)])
y_trans = Compose([EnsureType(), AsDiscrete(to_onehot=num_class)])

In [13]:
class MedNISTDataset(torch.utils.data.Dataset):
    def __init__(self, image_files, labels, transforms):
        self.image_files = image_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index]), self.labels[index]


test_ds = MedNISTDataset(test_x, test_y, val_transforms)
test_loader = torch.utils.data.DataLoader(
    test_ds, batch_size=300)

Define testing metric:

In [14]:
auc_metric = ROCAUCMetric()

To test the federated model we need to create a model instance and assign to it the model parameters estimated at the last federated optimization round.

In [15]:
model = exp.model_instance
model.load_state_dict(exp.aggregated_params[rounds - 1]['params'])

<All keys matched successfully>

Compute the testing performance:

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

y_true = []
y_pred = []
with torch.no_grad():
    for test_data in test_loader:
        test_images, test_labels = (
            test_data[0].to(device),
            test_data[1].to(device),
        )
        pred = model(test_images).argmax(dim=1)
        for i in range(len(pred)):
            y_true.append(test_labels[i].item())
            y_pred.append(pred[i].item())


In [17]:
print(classification_report(
    y_true, y_pred, target_names=class_names, digits=4))

              precision    recall  f1-score   support

   AbdomenCT     0.9802    0.9900    0.9851       100
   BreastMRI     0.9885    1.0000    0.9942        86
         CXR     1.0000    0.9902    0.9951       102
     ChestCT     0.9895    1.0000    0.9947        94
        Hand     0.9902    1.0000    0.9951       101
      HeadCT     1.0000    0.9744    0.9870       117

    accuracy                         0.9917       600
   macro avg     0.9914    0.9924    0.9919       600
weighted avg     0.9918    0.9917    0.9917       600



In spite of the relatively small training performed on the data shared in the 3 nodes, the performance of the federated model seems pretty good. Well done! 